In [ ]:
import numpy as np
import pandas as pd
import os
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
import xarray as xr
import re

In [ ]:
#User inputs
#name of the csv file generated from Excel Generation.ipynb
#NOTE: Excel Generation generates a CSV post change on 8/16/22, code has been adjusted here to read CSV files
#NOTE: These files should be in the working directory of the .ipynb file
fileName = "xxxx"

In [ ]:
#pandas dataframe created 
#reads csv file for this
data = pd.read_csv(fileName + ".csv")

In [ ]:
#format fileName into strings for use later. 
#index _ in string.

char = "_"
indices = [i.start() for i in re.finditer(char,fileName)]

#grab strings
locationName = fileName[:indices[1]]
startDate = fileName[indices[1]+1:indices[2]]
endDate = fileName[indices[2]+1:]

#format locationName into title format
locationName = locationName.replace("_"," ").title()

In [ ]:
#converts pandas dataframe to xarray for holoviews wrapper contour plot
time = pd.date_range(startDate, endDate)
depth = np.arange(3000)

ds = xr.Dataset(
    data_vars=dict(
        SoundSpeed=(["depth","time"], data)
    ), 
    coords=dict(
        time=(["time"], time), 
        depth=(["depth"], depth),
    ),
    attrs=dict(description="Sound Data"),
)

#holoviews wrapper
hv_ds = hv.Dataset(ds)[:,:]
soundspeed = hv_ds.to(hv.Image, kdims=["time","depth"])

In [ ]:
#holoviews tap reference: http://holoviews.org/reference/streams/bokeh/Tap.html

#Stream that uses the sound speed contour plot 
#x= is for initial date it will display
posxy = hv.streams.Tap(source=soundspeed, x=startDate)

#declares function that generates scatter plot with x date and title
def tap_date(x, y):
    date = pd.to_datetime(x)
    strDate = str(date.date())
    return hv.Scatter(data[strDate], label=f'{strDate}: Speed of Sound [m/s] vs Depth[m]')

#create dynamic map that takes in the tap stream and creates scatters plots based on date value from click
tap_dmap = hv.DynamicMap(tap_date, streams=[posxy])

#Generates 2 plots, contour plot & scatter plot next to it
(soundspeed + tap_dmap).opts(
    opts.Image(
        title=locationName + " Speed of Sound with Depth vs Time",
        width=600, height=500,
        tools=['hover'],
       
        ylabel='Depth [m]',
        invert_yaxis=True,
        ylim=(0,250),
       
        xlabel='Date',
       
        cmap='viridis',
        colorbar=True,
    ),

    opts.Scatter(
        framewise=True,
        width=375, height=500,
        tools=['hover'],
        invert_axes=True,
       
        ylabel='Speed of Sound [m/s]',
        yaxis='right',
        invert_yaxis=True,
        ylim=(1460, 1520),
       
        xlabel='Depth',
        xlim=(0,200),
    ),
)